________________
### 0. Delete Database file 

In [1]:
import os
import sqlite3

# Close any existing connections
conn = sqlite3.connect('reforestation.db')
conn.close()

# Delete the file completely
if os.path.exists('reforestation.db'):
    os.remove('reforestation.db')
    print ("Database file deleted completely")
else:
    print("Database file did not exist.")

Database file deleted completely


_______
### 1. Create Database file and table structure

In [3]:
import sqlite3

# Create a sql database file
conn = sqlite3.connect('reforestation.db')
cursor=conn.cursor()

# Create a table for the preliminary data
cursor.execute('''
    CREATE TABLE PRELIMINAR (
                ID INTEGER PRIMARY KEY AUTOINCREMENT,
                GENERAL_LOCATION TEXT,
                SPECIFIC_LOCATION TEXT,
                AREA_M2 REAL,
                PERIMETER_M REAL,
                TREES_BY_AREA INTEGER,
                TREES_BY_PERIMETER INTEGER
    )
''')

______
### 2. Data insertion

#### Single line

In [6]:
import sqlite3

# Connect to the existing database file
conn = sqlite3.connect('reforestation.db')
cursor=conn.cursor()

# Insert the second line
cursor.execute('''
    INSERT INTO PRELIMINAR
                (GENERAL_LOCATION, SPECIFIC_LOCATION, AREA_M2, PERIMETER_M)
            VALUES
                ('Parque lluvia de oro', 'Frente a Torcaza I', 893.16, 144.64)
''')

conn.commit()
conn.close()



#### Multiple lines

In [4]:
import sqlite3

# Insert multiple rows to th existing database
# list of tupples to insert
location_data = [
    ('Parque lluvia de oro', 'Rotonda', 1123.25, 122.22,),
    ('Parque lluvia de oro', 'Frente a Torcaza I', 893.16, 144.64),
    ('Parque lluvia de oro', 'Frente a Torcaza II', 850.62, 135.55),
    ('Parque lluvia de oro', 'Cancha de Futbol', 1658.05, 163.8 ),
    ('Parque lluvia de oro', 'Frente a Gorrión I', 539.14, 97.81 ),
    ('Parque lluvia de oro', 'Frente a Gorrión II', 2098.78, 188.02),
    ('Parque lluvia de oro', 'Frente a Gorrión III',2398.02,208.31),
    ('Parque lluvia de oro', 'Frente a Gorrión IV',1424.44, 152.78),
    ('Parque lluvia de oro', 'Frente a Tórtola I',2076.92, 194.4),
    ('Parque lluvia de oro', 'Frente a Tórtola II',2326.86, 205.78),
    ('Parque lluvia de oro', 'Servidumbre I',524.58,95.83),
    ('Parque lluvia de oro', 'Servidumbre II',472.91,83.13),
    ('Parque lluvia de oro', 'Servidumbre III',596.44,95.61),
    ('Parque lluvia de oro', 'Servidumbre IV',777.23,124.58),
    ('Parque lluvia de oro', 'Servidumbre V', 3954.72, 270.59),
]

# Single SQL command with parameterized query
insert_query = ''' 
    INSERT INTO PRELIMINAR 
        (GENERAL_LOCATION, SPECIFIC_LOCATION, AREA_M2, PERIMETER_M)
    VALUES
        (?,?,?,?)
''' 

# Excecute in transaction
with sqlite3.connect('reforestation.db') as conn:
    conn.executemany(insert_query, location_data)


______
### 5. Update data

In [10]:
import sqlite3

conn = sqlite3.connect('reforestation.db')
cursor = conn.cursor()

cursor.execute('''
    UPDATE PRELIMINAR
    SET
        TREES_BY_AREA = FLOOR(AREA_M2/16),
        TREES_BY_PERIMETER = FLOOR(PERIMETER_M/4)
    WHERE
        TREES_BY_AREA IS NULL
        OR TREES_BY_PERIMETER IS NULL
''')

conn.commit()
conn.close()

In [9]:
import sqlite3

conn = sqlite3.connect('reforestation.db')
cursor = conn.cursor()

# Use update set specific columns to null 
cursor.execute('''
    UPDATE PRELIMINAR
    SET 
        TREES_BY_AREA = NULL,
        TREES_BY_PERIMETER = NULL
''')

conn.commit()
conn.close()

In [3]:
import sqlite3
from IPython.display import display, HTML

def display_query_results(query):
    """Universal function to display SQL results in scrollable HTML table"""
    # Base CSS for all tables
    css = """
    <style>
        .scroll-table {
            max-height: 300px;
            overflow: auto;
            margin: 10px 0;
        }
        .data-table {
            border-collapse: collapse;
            width: 100%;
        }
        .data-table td, .data-table th {
            border: 1px solid #ddd;
            padding: 8px;
            text-align: right;
        }
        .data-table th {
            background: #f0f8ff;
            position: sticky;
            top: 0;
        }
    </style>
    """
    
    try:
        with sqlite3.connect('reforestation.db') as conn:
            cursor = conn.cursor()
            cursor.execute(query)
            
            # Get results and column names
            results = cursor.fetchall()
            columns = [desc[0] for desc in cursor.description]
            
            # Build HTML table
            html_table = f"{css}<div class='scroll-table'><table class='data-table'><tr>"
            html_table += "".join(f"<th>{col}</th>" for col in columns) + "</tr>"
            
            for row in results:
                html_table += "<tr>" + "".join(f"<td>{format_value(val)}</td>" for val in row) + "</tr>"
            
            html_table += "</table></div>"
            display(HTML(html_table))
            
    except Exception as e:
        print(f"Error: {e}")

def format_value(value):
    """Universal formatting for different data types"""
    if isinstance(value, (int, float)):
        return f"{value:,.2f}" if value % 1 else f"{value:,.0f}"
    return str(value)

# Usage example:
display_query_results('''
    SELECT 
        SUM(TREES_BY_AREA) AS TOTAL_AREA_TREES,
        SUM(TREES_BY_PERIMETER) AS TOTAL_PERIMETER_TREES
    FROM PRELIMINAR
''')



TOTAL_AREA_TREES,TOTAL_PERIMETER_TREES
"1,350",563
